# Condo Ownership vs Stock Index Investment

In [242]:
import numpy as np

## Yearly Expenditures for Condo Ownership (3494 hotel-de-ville)

(Property Tax for [Plateau Mont-Royale](http://ville.montreal.qc.ca/pls/portal/docs/PAGE/SERVICE_FIN_EN/MEDIA/DOCUMENTS/2019_PLATEAU_ANG.PDF)) * (Most recent property evaluation for 3494 hotel-de-ville)


In [243]:
plateau_tax_rate = 0.006519 + 0.000025 + 0.001083 + 0.000036 + 0.000591 + 0.000315
municipal_evaluation = 359000
property_tax = plateau_tax_rate * municipal_evaluation

In [244]:
condo_fees = 2400
insurance = 500
maintenance = 500

In [245]:
annual_expense = sum([
    condo_fees,
    insurance,
    maintenance,
    property_tax
])
annual_expense

6476.270999999999

## One-time Expenses for Condo Ownership

In [246]:
welcome_tax = 4500
notary = 1500
title_insurance = 400
inspection = 400

initial_expense = sum([welcome_tax, notary, title_insurance, inspection])
initial_expense

6800

## Mortgage Analysis

In [247]:
interest = 0.03
nper = 25
per = np.arange(nper) + 1
price = 387000
resale_price = 400000
downpayment = 20000

### CMHC Insurance

Interest rates [listed here](https://www.cmhc-schl.gc.ca/en/finance-and-investing/mortgage-loan-insurance/mortgage-loan-insurance-homeownership-programs/cmhc-mortgage-loan-insurance-cost). Mortgage insurance is covered by the lender for loan to value of under 80% (i.e., if your downpayment is at least 20%, this section is of no concern to you).

In [248]:
def mortgage_insurance_rate(p, downpayment):
    ltv = (p - downpayment) / p
    assert ltv > 0
    assert ltv <= 1
    
    if ltv >= 0.95:
        return 0.04
    elif ltv >= 0.90:
        return 0.031
    elif ltv >= 0.85:
        return 0.028
    elif ltv >= 0.80:
        return 0.024
    else:
        return 0
    
mortgage_insurance_rate(price, downpayment)

0.031

In [249]:
initial_principal = price + mortgage_insurance_rate(price, downpayment) * price - downpayment
ppmt = -np.ppmt(interest, per, nper, initial_principal)
ipmt = -np.ipmt(interest, per, nper, initial_principal)
pmt = ipmt + ppmt

In [250]:
fmt = '{0:2d} {4:8.2f} {1:8.2f} {2:8.2f} {3:8.2f}'
p = initial_principal
for payment in per:
    index = payment - 1
    p = p - ppmt[index]
    print(fmt.format(payment, ppmt[index], ipmt[index], p, ppmt[index] + ipmt[index]))

# Monthly mortgage payment
mrtg_monthly_payment = (ppmt[0] + ipmt[0]) / 12
mrtg_monthly_payment

 1 21764.99 10395.08 11369.91 368601.92
 2 21764.99 10706.93 11058.06 357894.99
 3 21764.99 11028.14 10736.85 346866.84
 4 21764.99 11358.99 10406.01 335507.86
 5 21764.99 11699.76 10065.24 323808.10
 6 21764.99 12050.75  9714.24 311757.36
 7 21764.99 12412.27  9352.72 299345.09
 8 21764.99 12784.64  8980.35 286560.45
 9 21764.99 13168.18  8596.81 273392.27
10 21764.99 13563.22  8201.77 259829.05
11 21764.99 13970.12  7794.87 245858.93
12 21764.99 14389.22  7375.77 231469.71
13 21764.99 14820.90  6944.09 216648.81
14 21764.99 15265.53  6499.46 201383.28
15 21764.99 15723.49  6041.50 185659.79
16 21764.99 16195.20  5569.79 169464.59
17 21764.99 16681.05  5083.94 152783.54
18 21764.99 17181.48  4583.51 135602.05
19 21764.99 17696.93  4068.06 117905.12
20 21764.99 18227.84  3537.15 99677.28
21 21764.99 18774.67  2990.32 80902.61
22 21764.99 19337.91  2427.08 61564.70
23 21764.99 19918.05  1846.94 41646.65
24 21764.99 20515.59  1249.40 21131.06
25 21764.99 21131.06   633.93     0.00


1813.749236684692

### Gross Debt Service (GDS)

A GDS ratio is the percentage of your income needed to pay all of your monthly housing costs, including principal, interest, taxes, and heat (PITH). You’ll also need to include 50 per cent of your condo fees, if applicable.

### Total Debt Service Ratio

Includes other debt obligations (credit cards, lines of credit, car loans, etc.)

You can include up to 50% of your rental income, and in this case, remove heating expenses from the calculation.

Canadian gov [reference](https://www.cmhc-schl.gc.ca/en/finance-and-investing/mortgage-loan-insurance/calculating-gds-tds)

In [251]:
gross_annual_income = 70000
monthly_heating = 200
rental_income = 2200 * 12
gds = sum([pmt[0], condo_fees, property_tax]) / (gross_annual_income + rental_income / 2)
print("GDS: {}".format(gds * 100))


GDS: 32.741901250259986


### Comparative Yearly Return

Condominium appreciation 
[Plateau Mont Royale centris](https://www.centris.ca/en/tools/real-estate-statistics/montreal-island/le-plateau-mont-royal-montreal) -- 8% last year

[Montreal shupilov.com](https://news.shupilov.com/blog/average-real-estate-prices-and-appreciation-rates-in-montreal/) -- 3% per year

[fciq.ca](https://www.fciq.ca/pdf/mot_economiste/me_052014_an.pdf)



In [252]:
initial_capital = annual_expense + initial_expense + downpayment
total_yearly_cost = annual_expense + pmt[0]
yearly_capital = total_yearly_cost - rental_income

appreciated_value = np.fv(0.03, per, 0, -resale_price)

cumulative_capital = initial_capital + np.matmul([yearly_capital]*nper, np.triu(np.ones(nper*nper).reshape((nper, nper))))
cumulative_principal_paid = np.matmul(ppmt, np.triu(np.ones(nper*nper).reshape((nper, nper))))
cumulative_principal_left = initial_principal - cumulative_principal_paid

(initial_capital, yearly_capital)

(33276.271, 1841.2618402163062)

In [253]:
# TODO: add (varying per year) (annual_expense + ipmt[0] - rental_income * 12) compounded by 1.07
# This would be the lost opportunity cost that you'd be otheriwse adding to your stock investments
# (e.g., what is the added capital beyond your initial investment that you must use to sustain your mortgage?)
stocks = np.fv(0.07, per, -yearly_capital, -initial_capital)
stocks

array([ 37446.87181022,  41909.41467715,  46684.33554476,  51793.50087311,
        57260.30777445,  63109.79115888,  69368.73838021,  76065.81190705,
        83231.68058075,  90899.16006162,  99103.36310615, 107881.8603638 ,
       117274.85242948, 127325.35393976, 138079.39055576, 149586.20973488,
       161898.50625654, 175072.66353472, 189169.01182236, 204252.10449014,
       220391.01364467, 237659.64644001, 256137.08353103, 275907.94121842,
       297062.75894392])

In [254]:
realestate = appreciated_value - cumulative_capital - cumulative_principal_left
realestate

array([  8280.548     ,  29506.21942521,  51423.89884838,  74054.34650945,
        97418.94545556, 121539.72022525, 146439.35609325, 172141.21889249,
       198669.37543092, 226048.61452071, 254304.46863839, 283463.23623482,
       313552.00471434, 344598.67410345, 376631.98142945, 409681.52583043,
       443777.79441865, 478952.18891972, 515237.05311102, 552665.70108328,
       591272.44634991, 631092.63182974, 672162.66072917, 714520.0283508 ,
       758203.35485628])

In [255]:
#calculate roi

roi_realestate = (realestate - cumulative_capital) / cumulative_capital
roi_realestate

array([-0.76420473, -0.20164552,  0.32535629,  0.8221443 ,  1.29315039,
        1.7420845 ,  2.17207899,  2.58579985,  2.98553322,  3.37325346,
        3.75067719,  4.11930652,  4.48046399,  4.83532095,  5.18492082,
        5.53019828,  5.87199515,  6.21107369,  6.5481277 ,  6.88379192,
        7.21865004,  7.55324139,  7.88806682,  8.22359368,  8.56026005])

In [256]:
first_year = rental_income * 12 - annual_expense - initial_expense + ppmt[0] - ipmt[0]
def net(year):
    if year == 1:
        return first_year
    return net(year - 1) + rental_income * 12 - annual_expense + ppmt[year] - ipmt[year]

In [257]:
np.fv(0.07, per, -80000, -50000)

array([ 133500.        ,  222845.        ,  318444.15      ,
        420735.2405    ,  530186.707335  ,  647299.77684845,
        772610.76122784,  906693.51451379, 1050162.06052976,
       1203673.40476684, 1367930.54310052, 1543685.68111756,
       1731743.67879578, 1932965.73631149, 2148273.33785329,
       2378652.47150302, 2625158.14450824, 2888919.21462381,
       3171143.55964748, 3473123.6088228 , 3796242.2614404 ,
       4141979.21974123, 4511917.76512312, 4907752.00868173,
       5331294.64928946])

In [258]:
realestate / stocks

array([0.2211279 , 0.70404752, 1.10152363, 1.42979998, 1.70133465,
       1.9258457 , 2.1110281 , 2.26305635, 2.38694418, 2.48680642,
       2.56605286, 2.62753382, 2.67365081, 2.70644191, 2.72764806,
       2.73876534, 2.7410864 , 2.73573372, 2.72368634, 2.70580175,
       2.68283374, 2.65544715, 2.62423016, 2.58970447, 2.55233392])